# Getting started with mne.Report
---

```mne.Report```는 데이터에 대한 HTML 문서를 만들 때 사용된다.   
```mne.Report``` 는 ```Jupyter notebook``` 과 다르게 파이썬 환경이 필요하지 않다.

In [ ]:
# 예시 데이터
from pathlib import Path
import tempfile
import numpy as np
import scipy.ndimage
import matplotlib.pyplot as plt
import mne

data_path = Path(mne.datasets.sample.data_path(verbose=False))
sample_dir = data_path / "MEG" / "sample"
subjects_dir = data_path / "subjects"

* MNE filename conventions

<table class="table-bordered midvalign table">
<thead>
<tr class="row-odd"><th class="head"><p>Data object</p></th>
<th class="head"><p>Filename convention (ends with)</p></th>
</tr>
</thead>
<tbody>
<tr class="row-even"><td><p><a class="reference internal" href="../../generated/mne.io.Raw.html#mne.io.Raw" title="mne.io.Raw"><code class="xref py py-obj docutils literal notranslate"><span class="pre">Raw</span></code></a></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">-raw.fif(.gz)</span></code>, <code class="docutils literal notranslate"><span class="pre">-raw_sss.fif(.gz)</span></code>,
<code class="docutils literal notranslate"><span class="pre">-raw_tsss.fif(.gz)</span></code>,
<code class="docutils literal notranslate"><span class="pre">_meg.fif(.gz)</span></code>, <code class="docutils literal notranslate"><span class="pre">_eeg.fif(.gz)</span></code>,
<code class="docutils literal notranslate"><span class="pre">_ieeg.fif(.gz)</span></code></p></td>
</tr>
<tr class="row-odd"><td><p>events</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">-eve.fif(.gz)</span></code></p></td>
</tr>
<tr class="row-even"><td><p><a class="reference internal" href="../../generated/mne.Epochs.html#mne.Epochs" title="mne.Epochs"><code class="xref py py-obj docutils literal notranslate"><span class="pre">Epochs</span></code></a></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">-epo.fif(.gz)</span></code></p></td>
</tr>
<tr class="row-odd"><td><p><a class="reference internal" href="../../generated/mne.Evoked.html#mne.Evoked" title="mne.Evoked"><code class="xref py py-obj docutils literal notranslate"><span class="pre">Evoked</span></code></a></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">-ave.fif(.gz)</span></code></p></td>
</tr>
<tr class="row-even"><td><p><a class="reference internal" href="../../generated/mne.Covariance.html#mne.Covariance" title="mne.Covariance"><code class="xref py py-obj docutils literal notranslate"><span class="pre">Covariance</span></code></a></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">-cov.fif(.gz)</span></code></p></td>
</tr>
<tr class="row-odd"><td><p><a class="reference internal" href="../../generated/mne.Projection.html#mne.Projection" title="mne.Projection"><code class="xref py py-obj docutils literal notranslate"><span class="pre">Projection</span></code></a></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">-proj.fif(.gz)</span></code></p></td>
</tr>
<tr class="row-even"><td><p><a class="reference internal" href="../../generated/mne.transforms.Transform.html#mne.transforms.Transform" title="mne.transforms.Transform"><code class="xref py py-obj docutils literal notranslate"><span class="pre">Transform</span></code></a></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">-trans.fif(.gz)</span></code></p></td>
</tr>
<tr class="row-odd"><td><p><a class="reference internal" href="../../generated/mne.Forward.html#mne.Forward" title="mne.Forward"><code class="xref py py-obj docutils literal notranslate"><span class="pre">Forward</span></code></a></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">-fwd.fif(.gz)</span></code></p></td>
</tr>
<tr class="row-even"><td><p><a class="reference internal" href="../../generated/mne.minimum_norm.InverseOperator.html#mne.minimum_norm.InverseOperator" title="mne.minimum_norm.InverseOperator"><code class="xref py py-obj docutils literal notranslate"><span class="pre">InverseOperator</span></code></a></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">-inv.fif(.gz)</span></code></p></td>
</tr>
</tbody>
</table>

#### Adding Raw data   

Raw data는 ```mne.Report.add_raw()``` method를 통해 추가할 수 있다.

In [ ]:
raw_path = sample_dir / "sample_audvis_filt-0-40_raw.fif"
raw = mne.io.read_raw(raw_path)
raw.pick_types(eeg=True, eog=True, stim=True).crop(tmax=60).load_data()

report = mne.Report(title="Raw example")
# This method also accepts a path, e.g., raw=raw_path
report.add_raw(raw=raw, title="Raw", psd=False)  # omit PSD plot
report.save("report_raw.html", overwrite=True)

```mne.Report.add_raw()```를 통해 생성된 ```Raw``` HTML 문서

<img src="../img/Getting started with mne.Report/Raw_Report_example.png" width=600>

#### Adding events

In [ ]:
events_path = sample_dir / "sample_audvis_filt-0-40_raw-eve.fif"
events = mne.find_events(raw=raw)
sfreq = raw.info["sfreq"]

report = mne.Report(title="Events example")
report.add_events(events=events_path, title="Events from Path", sfreq=sfreq)
report.add_events(events=events, title='Events from "events"', sfreq=sfreq)
report.save("report_events.html", overwrite=True)

```mne.Report.add_events()```로 생성된 ```events```에 관한 HTML 문서   

<img src="../img/Getting started with mne.Report/events_Report_example.png" width=600>

#### Adding Epochs


In [ ]:
event_id = {
    "auditory/left": 1,
    "auditory/right": 2,
    "visual/left": 3,
    "visual/right": 4,
    "face": 5,
    "buttonpress": 32,
}

metadata, _, _ = mne.epochs.make_metadata(
    events=events, event_id=event_id, tmin=-0.2, tmax=0.5, sfreq=raw.info["sfreq"]
)
epochs = mne.Epochs(raw=raw, events=events, event_id=event_id, metadata=metadata)

report = mne.Report(title="Epochs example")
report.add_epochs(epochs=epochs, title='Epochs from "epochs"')
report.save("report_epochs.html", overwrite=True)

```mne.Report.add_epochs()```로 생성된 ```Epochs``` HTML 문서   
```event_id```는 metadata를 생성하기 위해 필요한 파라미터이다.

<img src="../img/Getting started with mne.Report/Epochs_Report_example.png" width=600>

#### Adding Evoked


In [ ]:
evoked_path = sample_dir / "sample_audvis-ave.fif"
cov_path = sample_dir / "sample_audvis-cov.fif"

evokeds = mne.read_evokeds(evoked_path, baseline=(None, 0))
evokeds_subset = evokeds[:2]  # The first two
for evoked in evokeds_subset:
    evoked.pick("eeg")  # just for speed of plotting

report = mne.Report(title="Evoked example")
report.add_evokeds(
    evokeds=evokeds_subset,
    titles=["evoked 1", "evoked 2"],  # Manually specify titles
    noise_cov=cov_path,
    n_time_points=21,
)
report.save("report_evoked.html", overwrite=True)

```mne.Report.add_evokeds()```로 생성된 ```Evoked``` HTML 문서   
```n_time_points``` parameter를 통해 snapshot 간격을 최대 21조각으로 나눌 수 있다.   

<img src="../img/Getting started with mne.Report/Evoked_Report_example.png" width=600>

#### Adding Covariance

In [ ]:
cov_path = sample_dir / "sample_audvis-cov.fif"

report = mne.Report(title="Covariance example")
report.add_covariance(cov=cov_path, info=raw_path, title="Covariance")
report.save("report_cov.html", overwrite=True)

```mne.Report.add_covariance()```로 생성된 ```covariance``` HTML 문서

<img src="../img/Getting started with mne.Report/Covariance_Report_example.png" width=600>

#### Adding Projection vectors


In [ ]:
ecg_proj_path = sample_dir / "sample_audvis_ecg-proj.fif"
eog_proj_path = sample_dir / "sample_audvis_eog-proj.fif"

report = mne.Report(title="Projectors example")
report.add_projs(info=raw_path, title="Projs from info")
report.add_projs(info=raw_path, projs=ecg_proj_path, title="ECG projs from path")
report.add_projs(info=raw_path, projs=eog_proj_path, title="EOG projs from path")
report.save("report_projs.html", overwrite=True)

```mne.Report.add_projs()```로 생성된 ```projection vectors``` HTML 문서

<img src="../img/Getting started with mne.Report/Projectors_Report_example.png" width=600>

#### Adding ICA

In [ ]:
ica = mne.preprocessing.ICA(
    n_components=5,  # fit 5 ICA components
    fit_params=dict(tol=0.01),  # assume very early on that ICA has converged
)

ica.fit(inst=raw)

# create epochs based on EOG events, find EOG artifacts in the data via pattern
# matching, and exclude the EOG-related ICA components
eog_epochs = mne.preprocessing.create_eog_epochs(raw=raw)
eog_components, eog_scores = ica.find_bads_eog(
    inst=eog_epochs,
    ch_name="EEG 001",  # a channel close to the eye
    threshold=1,  # lower than the default threshold
)
ica.exclude = eog_components

report = mne.Report(title="ICA example")
report.add_ica(
    ica=ica,
    title="ICA cleaning",
    picks=[0, 1],  # only plot the first two components
    inst=raw,
    eog_evoked=eog_epochs.average(),
    eog_scores=eog_scores,
    n_jobs=None,  # could be increased!
)
report.save("report_ica.html", overwrite=True)

```mne.Report.add_ica```로 생성된 ```ICA``` HTML 문서   
```inst``` parameter에 따라 ```ICA``` data를 ```raw`` data 등 여러 데이터와 함께 띄울 지 혼자 띄울 지 결정할 수 있다.   


<img src="../img/Getting started with mne.Report/ICA_Report_example.png" width=600>

#### Adding MRI with BEM

In [ ]:
report = mne.Report(title="BEM example")
report.add_bem(
    subject="sample", subjects_dir=subjects_dir, title="MRI & BEM", decim=20, width=256
)
report.save("report_mri_and_bem.html", overwrite=True)

```mne.Report.add_bem()```로 생성된 MRI BEM(단면)에 관한 HTML 문서   

```decim```, ```width``` parameter를 조절하여 용량을 줄일 수 있다.   

<img src="../img/Getting started with mne.Report/BEM_Report_example.png" width=600>

#### Adding coregistration


In [ ]:
trans_path = sample_dir / "sample_audvis_raw-trans.fif"

report = mne.Report(title="Coregistration example")
report.add_trans(
    trans=trans_path,
    info=raw_path,
    subject="sample",
    subjects_dir=subjects_dir,
    alpha=1.0,
    title="Coregistration",
)
report.save("report_coregistration.html", overwrite=True)

```mne.Reoirt.add_trans()```로 생성된 ```Coregistration``` HTML 문서   

<img src="../img/Getting started with mne.Report/Coregistration_Report_exmaple.png" width=600>

#### Adding a Forward solution

In [ ]:
fwd_path = sample_dir / "sample_audvis-meg-oct-6-fwd.fif"

report = mne.Report(title="Forward solution example")
report.add_forward(forward=fwd_path, title="Forward solution")
report.save("report_forward_sol.html", overwrite=True)

```mne.Report.add_forward()```로 생성된 ```Forward solution``` HTML 문서

<img src="../img/Getting started with mne.Report/Forward_Report_example.png" width=600>

#### Adding an InverseOperator


In [ ]:
inverse_op_path = sample_dir / "sample_audvis-meg-oct-6-meg-inv.fif"

report = mne.Report(title="Inverse operator example")
report.add_inverse_operator(inverse_operator=inverse_op_path, title="Inverse operator")
report.save("report_inverse_op.html", overwrite=True)

```mne.Report.add_inverse_operator()```로 생성된 ```Inverse operator``` HTML 문서

<img src="../img/Getting started with mne.Report/Inverse_Report_example.png" width=600>

#### Adding a SourceEstimate


In [ ]:
stc_path = sample_dir / "sample_audvis-meg"

report = mne.Report(title="Source estimate example")
report.add_stc(
    stc=stc_path,
    subject="sample",
    subjects_dir=subjects_dir,
    title="Source estimate",
    n_time_points=25,  # few for speed
)
report.save("report_inverse_sol.html", overwrite=True)

```mne.Report.add_stc()```로 생성된 ```Source estimate``` HTML 문서

<img src="../img/Getting started with mne.Report/STC_Report_example.png" width=600>

#### Adding source code (e.g., a Python script)

In [ ]:
mne_init_py_path = Path(mne.__file__)  # __init__.py in the MNE-Python root
mne_init_py_content = mne_init_py_path.read_text(encoding="utf-8")

report = mne.Report(title="Code example")
report.add_code(code=mne_init_py_path, title="Code from Path")
report.add_code(code=mne_init_py_content, title="Code from string")

report.save("report_code.html", overwrite=True)

```mne.Report.add_code()```로 생성된 ```Code``` HTML 문서

<img src="../img/Getting started with mne.Report/Code_Report_example.png" width=600>

#### Adding custom figures

In [ ]:
x = np.linspace(start=0, stop=10, num=100)
y = x**2

fig, ax = plt.subplots()
ax.plot(x, y, ls="--", lw=2, color="blue", label="my function")
ax.set_xlabel("x")
ax.set_ylabel("f(x)")
ax.legend()

report = mne.Report(title="Figure example")
report.add_figure(
    fig=fig,
    title="A custom figure",
    caption="A blue dashed line reaches up into the sky …",
    image_format="PNG",
)
report.save("report_custom_figure.html", overwrite=True)
plt.close(fig)

```mne.Report.add_figure()```를 통해 ```matplotlib``` 결과도 HTML 문서로 생성할 수 있다.

<img src="../img/Getting started with mne.Report/Figure_Report_example.png" width=600>

In [ ]:
# 여러 개의 plot을 한 section 안에 띄우기
fig_1, ax_1 = plt.subplots()
ax_1.plot([1, 2, 3])

fig_2, ax_2 = plt.subplots()
ax_2.plot([3, 2, 1])

section = "Section example"

report = mne.Report(title="Figure section example")
report.add_figure(fig=fig_1, title="Figure 1", section=section, tags="fig-1")
report.add_figure(fig=fig_2, title="Figure 2", section=section, tags="fig-2")
report.save("report_custom_figure_sections.html", overwrite=True)
plt.close(fig_1)
plt.close(fig_2)

<img src="../img/Getting started with mne.Report/Figure2_Report_example.png" width=600>

In [ ]:
mne_logo_path = Path(mne.__file__).parent / "icons" / "mne_icon-cropped.png"
fig_array = plt.imread(mne_logo_path)
rotation_angles = np.linspace(start=0, stop=360, num=17)

figs = []
captions = []
for angle in rotation_angles:
    # Rotate and remove some rounding errors to avoid Matplotlib warnings
    fig_array_rotated = scipy.ndimage.rotate(input=fig_array, angle=angle)
    fig_array_rotated = fig_array_rotated.clip(min=0, max=1)

    # Create the figure
    fig, ax = plt.subplots()
    ax.imshow(fig_array_rotated)
    ax.set_axis_off()

    # Store figure and caption
    figs.append(fig)
    captions.append(f"Rotation angle: {round(angle, 1)}°")

# can also be a MNEQtBrowser instance
figs.append(raw.plot())
captions.append("... plus a raw data plot")

report = mne.Report(title="Multiple figures example")
report.add_figure(fig=figs, title="Fun with figures! 🥳", caption=captions)
report.save("report_custom_figures.html", overwrite=True)
for fig in figs[:-1]:
    plt.close(fig)
figs[-1].close()
del figs

여러 개의 plot을 slider 요소를 통해서 띄울 수 있다.   
위 코드는 MNE 로고를 360도 회전하는 plot 여러 개를 slider를 통해 한 번에 나타낸 것이다.

<img src="../img/Getting started with mne.Report/Figure3_Report_example.png" width=600>

#### Adding image files


In [21]:
report = mne.Report(title="Image example")
report.add_image(
    image=mne_logo_path, title="MNE", caption="Powered by 🧠 🧠 🧠 around the world!"
)
report.save("report_custom_image.html", overwrite=True)

Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : bootstrap-table/bootstrap-table.min.js
Embedding : bootstrap-table/bootstrap-table.min.css
Embedding : bootstrap-table/bootstrap-table-copy-rows.min.js
Embedding : bootstrap-table/bootstrap-table-export.min.js
Embedding : bootstrap-table/tableExport.min.js
Embedding : bootstrap-icons/bootstrap-icons.mne.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
Saving report to : C:\Users\supli\Documents\Repo\MNE_tutorials\Inuductory_tutorials\report_custom_image.html


'C:\\Users\\supli\\Documents\\Repo\\MNE_tutorials\\Inuductory_tutorials\\report_custom_image.html'

```mne.Report.add_image()```를 통해 HTML 문서에 이미지를 추가할 수 있다.

<img src="../img/Getting started with mne.Report/Image_Report_example.png" width=600>

#### Working with tags

In [22]:
report = mne.Report(title="Tags example")
report.add_image(
    image=mne_logo_path, title="MNE Logo", tags=("image", "mne", "logo", "open-source")
)
report.save("report_tags.html", overwrite=True)

Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : bootstrap-table/bootstrap-table.min.js
Embedding : bootstrap-table/bootstrap-table.min.css
Embedding : bootstrap-table/bootstrap-table-copy-rows.min.js
Embedding : bootstrap-table/bootstrap-table-export.min.js
Embedding : bootstrap-table/tableExport.min.js
Embedding : bootstrap-icons/bootstrap-icons.mne.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
Saving report to : C:\Users\supli\Documents\Repo\MNE_tutorials\Inuductory_tutorials\report_tags.html


'C:\\Users\\supli\\Documents\\Repo\\MNE_tutorials\\Inuductory_tutorials\\report_tags.html'

모든 ```mne.Report.add_*()``` 함수의 ```tag``` parameter를 통해 키워드 태깅을 할 수 있다.

<img src="../img/Getting started with mne.Report/Tags_Report_example.png" width=600>

#### Editing a saved report   
HTML 파일을 다시 ```Report``` 객체로 읽어들이는 것은 불가능하기에 이를 위해서는 HDF5 파일로 저장해야 된다.

In [23]:
report = mne.Report(title="Saved report example", verbose=True)
report.add_image(image=mne_logo_path, title="MNE 1")
report.save("report_partial.hdf5", overwrite=True)

Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : bootstrap-table/bootstrap-table.min.js
Embedding : bootstrap-table/bootstrap-table.min.css
Embedding : bootstrap-table/bootstrap-table-copy-rows.min.js
Embedding : bootstrap-table/bootstrap-table-export.min.js
Embedding : bootstrap-table/tableExport.min.js
Embedding : bootstrap-icons/bootstrap-icons.mne.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
Saving report to : C:\Users\supli\Documents\Repo\MNE_tutorials\Inuductory_tutorials\report_partial.hdf5


'C:\\Users\\supli\\Documents\\Repo\\MNE_tutorials\\Inuductory_tutorials\\report_partial.hdf5'

In [24]:
# report 객체 읽어들어 수정
report_from_disk = mne.open_report("report_partial.hdf5")
report_from_disk.add_image(image=mne_logo_path, title="MNE 2")
report_from_disk.save("report_partial.hdf5", overwrite=True)

Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : bootstrap-table/bootstrap-table.min.js
Embedding : bootstrap-table/bootstrap-table.min.css
Embedding : bootstrap-table/bootstrap-table-copy-rows.min.js
Embedding : bootstrap-table/bootstrap-table-export.min.js
Embedding : bootstrap-table/tableExport.min.js
Embedding : bootstrap-icons/bootstrap-icons.mne.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
Overwriting existing file.
Saving report to : C:\Users\supli\Documents\Repo\MNE_tutorials\Inuductory_tutorials\report_partial.hdf5


'C:\\Users\\supli\\Documents\\Repo\\MNE_tutorials\\Inuductory_tutorials\\report_partial.hdf5'

In [25]:
# 최종적으로 html 파일로 저장
with mne.open_report("report_partial.hdf5") as report:
    report.add_image(image=mne_logo_path, title="MNE 3")
    report.save("report_final.html", overwrite=True)

Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : bootstrap-table/bootstrap-table.min.js
Embedding : bootstrap-table/bootstrap-table.min.css
Embedding : bootstrap-table/bootstrap-table-copy-rows.min.js
Embedding : bootstrap-table/bootstrap-table-export.min.js
Embedding : bootstrap-table/tableExport.min.js
Embedding : bootstrap-icons/bootstrap-icons.mne.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
Saving report to : C:\Users\supli\Documents\Repo\MNE_tutorials\Inuductory_tutorials\report_final.html
Overwriting existing file.
Saving report to : C:\Users\supli\Documents\Repo\MNE_tutorials\Inuductory_tutorials\report_partial.hdf5


<img src="../img/Getting started with mne.Report/HDF5_Report_example.png" width=600>

#### Adding an entire folder of files

```mne.Report.parse_folder()```를 통해 여러 파일이 저장된 폴더를 한 번에 추가할 수 있다.   
더 자세한 내용은 [여기](https://mne.tools/stable/auto_tutorials/intro/70_report.html#adding-an-entire-folder-of-files)를 참고.

In [28]:
report = mne.Report(title="parse_folder example")
report.parse_folder(
    data_path=data_path, pattern="*raw.fif", render_bem=False, raw_butterfly=False
)
report.save("report_parse_folder_basic.html", overwrite=True)

Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : bootstrap-table/bootstrap-table.min.js
Embedding : bootstrap-table/bootstrap-table.min.css
Embedding : bootstrap-table/bootstrap-table-copy-rows.min.js
Embedding : bootstrap-table/bootstrap-table-export.min.js
Embedding : bootstrap-table/tableExport.min.js
Embedding : bootstrap-icons/bootstrap-icons.mne.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
Opening raw data file C:\Users\supli\mne_data\MNE-sample-data\MEG\sample\ernoise_raw.fif...
Isotrak not found
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 19800 ... 85867 =     32.966 ...   142.965 secs
Ready.
Opening raw data file C:\Users\supli\mne_data\MNE-sample-data\MEG\sample\sample_audvis_filt-0-40_raw.fif...
    Read a total of 4 projection items:
        PCA-v1 (1 x 1

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


'C:\\Users\\supli\\Documents\\Repo\\MNE_tutorials\\Inuductory_tutorials\\report_parse_folder_basic.html'

<img src="../img/Getting started with mne.Report/Folder_Report_example.png" width=600>

#### Adding custom HTML (e.g., a description text)

In [30]:
report = mne.Report(title="Report on hypothesis 1")

my_html = """
<p>We have the following hypothesis:</p>
<ol>
<li>There is a difference between images showing man-made vs. natural
environments</li>
<li>This difference manifests itself most strongly in the amplitude of the
N1 ERP component</li>
</ol>
<p>Below we show several plots and tests of the data.</p>
"""

report.add_html(title="Hypothesis", html=my_html)
report.save("report_add_html.html", overwrite=True)

Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : bootstrap-table/bootstrap-table.min.js
Embedding : bootstrap-table/bootstrap-table.min.css
Embedding : bootstrap-table/bootstrap-table-copy-rows.min.js
Embedding : bootstrap-table/bootstrap-table-export.min.js
Embedding : bootstrap-table/tableExport.min.js
Embedding : bootstrap-icons/bootstrap-icons.mne.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
Saving report to : C:\Users\supli\Documents\Repo\MNE_tutorials\Inuductory_tutorials\report_add_html.html


'C:\\Users\\supli\\Documents\\Repo\\MNE_tutorials\\Inuductory_tutorials\\report_add_html.html'

```mne.Report.add_html()```을 통해 HTML 문서를 직접 작성할 수 있다.

<img src="../img/Getting started with mne.Report/HTML_Report_example.png" width=600>